# Scripts de pré-processamento

In [1]:
from sklearn.model_selection import train_test_split

from preprocess_funcs import *

In [2]:
def save_current(label, X_train, y_train, X_test=None, y_test=None, test=True):
    df_train = pd.concat([X_train, y_train], axis=1)
    df_train_blcd = hybrid_balancing(X_train, y_train, auto=True)

    df_train.to_csv(f'../data/preprocessed/train-validation/{label}.csv')
    df_train_blcd.to_csv(f'../data/preprocessed/train-validation/balanced/{label}.csv')
    if test == False:
        return
    df_test = pd.concat([X_test, y_test], axis=1)
    df_test.to_csv(f'../data/preprocessed/test/{label}.csv')


In [3]:
df = pd.read_csv('../data/students.csv')

## Remoção de colunas com baixa importancia (consultar arquivos no diretório 'exploratory analysis' para mais detalhes)

In [4]:
drop_cols = [
    'Nacionality', 
    'Educational special needs', 
    'International',
    'Curricular units 1st sem (credited)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (grade)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (without evaluations)'
    ]

df_drp = df.drop(columns=drop_cols)

X = df_drp.loc[:, df_drp.columns != 'Target']
y = df_drp['Target']

y = y.apply(lambda x: 'Graduate_or_Enrolled' if x != 'Dropout' else x)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

save_current('drp', X_train, y_train, X_test, y_test)

c:\Users\Felipe Castro\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Felipe Castro\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


## One-hot-encoding e Normalização

In [5]:
cat_drp_nb_cols = [
    'Marital status', 
    'Application mode', 
    'Course',
    'Previous qualification',
    'Mother\'s qualification', 
    'Father\'s qualification',
    'Mother\'s occupation', 
    'Father\'s occupation', 
    'Application order'
    ]

cat_drp_b_cols = [
    'Daytime/evening attendance', 
    'Displaced',
    'Debtor', 
    'Tuition fees up to date',
    'Gender', 
    'Scholarship holder',
]

num_drp_cols = [
    'Age at enrollment',
    'Curricular units 1st sem (approved)',
    'Curricular units 2nd sem (approved)',
    'Unemployment rate',
    'Inflation rate', 
    'GDP'
]

cat_vars = X[cat_drp_nb_cols] # Ex.: 'Marital status' = {0, 1, 2, 3, ...}
bin_var = X[cat_drp_b_cols] # Ex.: 'International' = {0, 1}
num_vars = X[num_drp_cols] # Ex.: 'Age at enrollment' = {17, 18, 19, ...}

cat_vars_OHE = one_hot_encoding(cat_vars, cat_drp_nb_cols)
num_vars_scaled = scaling(num_vars, 'minmax')

# apenas one hot encoding
XOHE = pd.concat([cat_vars_OHE, bin_var, num_vars, y], axis=1)

X = XOHE.loc[:, XOHE.columns != 'Target']
y = XOHE['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

save_current('drp_XOHE', X_train, y_train, X_test, y_test)


# apenas normalização
scaled = pd.concat([cat_vars, bin_var, num_vars_scaled, y], axis=1)

X = scaled.loc[:, scaled.columns != 'Target']
y = scaled['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

save_current('drp_scld', X_train, y_train, X_test, y_test)

# ambos
XOHE_scaled = pd.concat([cat_vars_OHE, bin_var, num_vars_scaled, y], axis=1) 

X = XOHE_scaled.loc[:, XOHE_scaled.columns != 'Target']
y = XOHE_scaled['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

save_current('drp_XOHE_scld', X_train, y_train, X_test, y_test)